In [7]:
import json 
path = '/home/gridsan/afogelson/osfm/paper_analysis_toolkit/data/foundation_models.json'

with open(path, 'r') as f:
    fm = json.load(f)
    mapping = {key: fm[key]['paperId'] for key in fm}

In [8]:
from collections import Counter

id_to_keys = {id: [] for id in {item for _, item in mapping.items()}}
for key, item in mapping.items():
    id_to_keys[item].append(key)

nonunique_id_to_keys = {key:item for key, item in id_to_keys.items() if len(item) > 1}

In [9]:
from typing import List
import regex as re

def manual_prefix_suffix(key):
    matches = re.findall('(.*)(?:\-|\_)\(?(\d+b)\)?', key)
    return matches[0] if matches else (None, None)

def extract_suffixes(id: str = None):
    keys =  list({key for key, item in mapping.items() if item == id})     
    stripped_keys = list(map(lambda s: re.findall('\d+_(.*)', s)[0], keys))
       
    names, suffixes = list(zip(*map(manual_prefix_suffix, stripped_keys)))
    if len(set(names)) == 1 and None not in names:
        model_name = next(iter(names))
    else:
        max_idx = 0
        for chars in zip(*stripped_keys):
            if len(set(chars)) == 1:
                max_idx += 1
            else:
                break
        
        if max_idx == 0:
            return id, None, keys
        
        formatter = lambda s: re.sub('\(*\)*', '', s)
        model_name =  formatter(next(iter(stripped_keys))[:max_idx])
        suffixes = [formatter(key[max_idx:]) for key in stripped_keys]
    
    return id, model_name, suffixes


In [10]:
results = [extract_suffixes(id) for id in nonunique_id_to_keys]
delineator_mapping = {id: (model_name, suffixes) for id, model_name, suffixes in results}
with open('output.txt', 'w') as f:
    json.dump(delineator_mapping, f, indent = 2)
    


In [11]:
print(id_to_keys['5a2304ba4e4401db2e0df8188a5f761646b52480'])

['55_true-regularization+finetune+dynamic-eval', '1058_true-regularization+finetune']
